In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import clear_output
import pandas as pd
import time
import random
import pickle
import lxml
import requests
import re 
import requests
from bs4 import BeautifulSoup
import re
import time
from recipe_scrapers import scrape_me
import requests
from bs4 import BeautifulSoup
import time


In [13]:
scraper = scrape_me("https://www.allrecipes.com/recipe/158968/spinach-and-feta-turkey-burgers/")
# scraper.title()
# scraper.instructions()
scraper.to_json()
# for a complete list of methods:


{'author': 'FoodieGeek',
 'canonical_url': 'https://www.allrecipes.com/recipe/158968/spinach-and-feta-turkey-burgers/',
 'category': 'Dinner',
 'cook_time': 15,
 'cuisine': 'Mediterranean Inspired',
 'description': 'These spinach and feta turkey burgers are moist and easy to make in one bowl with simple ingredients, shaped into patties, and cooked on a hot grill.',
 'host': 'allrecipes.com',
 'image': 'https://www.allrecipes.com/thmb/cpf6Rics5oHGq1TZ1df5fEaImwM=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/1360550-582be362ee99424bb4f363c2274a9d0d.jpg',
 'ingredient_groups': [{'ingredients': ['cooking spray',
    '2 pounds ground turkey',
    '1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry',
    '4 ounces feta cheese',
    '2 large eggs, beaten',
    '2 cloves garlic, minced'],
   'purpose': None}],
 'ingredients': ['cooking spray',
  '2 pounds ground turkey',
  '1 (10 ounce) box frozen chopped spinach, thawed and squeezed dry',
  '4 ounces feta cheese',
 

In [4]:
# help(scraper)
from recipe_scrapers import SCRAPERS
for site in list(SCRAPERS.keys()):
    if '.com' == site[-4:]:
        print(site, end = '\t')
        

abeautifulmess.com	acouplecooks.com	aflavorjournal.com	ahealthysliceoflife.com	alittlebityummy.com	aberlehome.com	abuelascounter.com	addapinch.com	afghankitchenrecipes.com	akispetretzikis.com	alexandracooks.com	allrecipes.com	allthehealthythings.com	altonbrown.com	amazingribs.com	ambitiouskitchen.com	americastestkitchen.com	ameessavorydish.com	archanaskitchen.com	averiecooks.com	bbc.com	bbcgoodfood.com	baking-sense.com	bakingmischief.com	barefootcontessa.com	barefeetinthekitchen.com	barefootinthepines.com	betterfoodguru.com	bettycrocker.com	beyondfrosting.com	biancazapatka.com	bigoven.com	bitsofcarey.com	blueapron.com	bluejeanchef.com	bodybuilding.com	bonappetit.com	bowlofdelicious.com	bongeats.com	breadtopia.com	brokenovenbaking.com	budgetbytes.com	cafedelites.com	cakemehometonight.com	cambreabakes.com	carlsbadcravings.com	cdkitchen.com	celebratingsweets.com	chefjeanpierre.com	chefsavvy.com	chefnini.com	closetcooking.com	colleenchristensennutrition.com	comidinhasdochef.com	cookeatshar

In [7]:
# need to look on each site, 
# find recipe index page 
# use developer tools to find where recipes are on page 
# ask chat gpt to get recipe links 
# identify how page indexes 
# save links 

In [3]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def get_recipe_links_m(url):
    """Scrapes all recipe links from a given page."""
    recipe_links = []
    
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        soup = soup.find("div", {"class":"block-category-listing__inner"})
        # Find all recipe links inside <article> tags
        for article in soup.find_all("a", class_="cat"):    # needs class update
            recipe_links.append(article['href'])
            # link_tag = article.find("a")                                 # needs class update
            # if link_tag and "href" in link_tag.attrs:
            #     recipe_links.append(link_tag["href"])

    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")

    return recipe_links
    
    
all_links = set()  # Use a set to avoid duplicates
url = "https://www.wellplated.com/recipe-index/"  # Start with the first page


print(f"Scraping: {url}")
response = requests.get(url, headers=HEADERS, timeout=10)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# Get recipe links from the current page
new_links = get_recipe_links_m(url)
all_links.update(new_links)
dirs = list(all_links)
print(len(dirs))

Scraping: https://www.wellplated.com/recipe-index/
22


In [12]:
def get_recipe_links(url):
    """Scrapes all recipe links from a given page."""
    recipe_links = []
    
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Find all recipe links inside <article> tags
        for article in soup.find_all("li", class_="listing-item"):    # needs class update
            link_tag = article.find("a")                                 # needs class update
            if link_tag and "href" in link_tag.attrs:
                recipe_links.append(link_tag["href"])

    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")

    return recipe_links


def find_pagination(soup):
    """Finds the next page link for pagination."""
    
    # # Find the pagination div
    next_page = soup.find("nav", class_ = "navigation pagination")
    next_page_tag = next_page.find("a", class_="next page-numbers")
    
    try:
        return next_page_tag['href']
    except:
        return None  # No next page found

def scrape_all_recipes(BASE_URL):
    """Finds and saves all recipe links from paginated index pages."""
    all_links = set()  # Use a set to avoid duplicates
    url = BASE_URL  # Start with the first page

    while url:
        print(f"Scraping: {url}")
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Get recipe links from the current page
        new_links = get_recipe_links(url)
        all_links.update(new_links)
        
        try:
            # Find the next page for pagination
            url = find_pagination(soup)
            # Optional delay to prevent rate-limiting
            time.sleep(2)
        except:
            return None
        
        

    # Save links to a file
    with open(OUTPUT_FILE, "a") as file:
        for link in all_links:
            file.write(link + "\n")
    
    print(f"✅ Scraping completed! {len(all_links)} links saved to {OUTPUT_FILE}")

# Run the scraper
OUTPUT_FILE = "wedishitup.txt"
scrape_all_recipes("https://wedishitup.com/category/all-recipes/")
# for directory in dirs:
#     scrape_all_recipes(directory)
    

Scraping: https://wedishitup.com/category/all-recipes/
Scraping: https://wedishitup.com/category/all-recipes/page/2/
Scraping: https://wedishitup.com/category/all-recipes/page/3/
Scraping: https://wedishitup.com/category/all-recipes/page/4/
Scraping: https://wedishitup.com/category/all-recipes/page/5/
Scraping: https://wedishitup.com/category/all-recipes/page/6/
Scraping: https://wedishitup.com/category/all-recipes/page/7/
Scraping: https://wedishitup.com/category/all-recipes/page/8/
Scraping: https://wedishitup.com/category/all-recipes/page/9/
Scraping: https://wedishitup.com/category/all-recipes/page/10/
Scraping: https://wedishitup.com/category/all-recipes/page/11/
Scraping: https://wedishitup.com/category/all-recipes/page/12/
Scraping: https://wedishitup.com/category/all-recipes/page/13/
Scraping: https://wedishitup.com/category/all-recipes/page/14/
Scraping: https://wedishitup.com/category/all-recipes/page/15/
Scraping: https://wedishitup.com/category/all-recipes/page/16/
Scraping